# Projekt IUM
## Etap 1
###### Marcel Kawski, Konrad Kulesza

# Opis zadania
---
### Definicja problemu biznesowego

„Niektórzy klienci nie mogą zdecydować się na zakup oglądając produkt na stronie –
ale pewnie, gdybyśmy wiedzieli, które to są osoby, to odpowiednia zniżka skłoniłaby
ich do zakupu”.

Wspomaganie sklepu internetowego w przyznawaniu zniżek w celu zwiększenia liczby sprzedanych produktów


### Biznesowe kryterium sukcesu
Na ten moment odsetek udanych sesji wynosi około 23%. Sukcesem będzie zwiększenie go o minimum 5 punktów procentowych.


### Kontekst naszego modelu
Nasz model będzie jednym komponentem całego, złożonego systemu. Wiemy, że inny zespół/model na podstawie m.in. naszych predykcji będzie próbował przyznawać takie wartości zniżek, aby klienci zostawiali w naszej firmie jak najwięcej pieniędzy. 

### Zadanie modelowania
Binarna klasyfikacja czy sesja danej osoby zakończy się zakupem(w zależności od przyznanej zniżki)

### Analityczne kryterium sukcesu
Naiwny klasyfikator stwierdzający, że nikt nie kupi będzie miał skuteczność ~77%.

p = (liczba sesji poprawnie sklasyfikowanych / liczba sesji)

Sukcesem będzie osiągnięcie:
**p >= 82%**


-----
-----
-----



# Analiza danych
---
### Czyszczenie oraz obróbka dostępnych danych 

#### users.jsonl
Dane są poprawne.
Wykorzystamy: 
- `user_id`,
- `city`
- z `name` wydobędziemy płeć użytkownika(imie kończy się na `a` -> kobieta)

#### products.jsonl
Atrybuty wymagające interwencji:
- `price` - niektóre wartości ujemne albo bardzo wysokie. Ze względu na duży zbiór danych postanowiliśmy 'wyrzucić' dane rekordy. Jednak, w przypadku stwierdzenia, że mamy za mało danych to przywrócimy dane z ujemnymi wartościami i zwyczajnie zanegujemy ich wartość
- `category_path` - rozbiliśmy ścieżkę na węzły i zakodowaliśmy ją za pomocą `one-hot encoding`. Dzięki temu nie straciliśmy informacji o podobieństwie danych produktów.

Dodatkowo wykorzystamy:
- `rating`
- `price`

#### sessions.jsonl
Wymagają interwencji:
- `user_id` czasami jest nullem.
- `product_id` czasami jest nullem.

Potraktowaliśmy braki jako `MCAR` i postanowiliśmy odrzucić błędne rekordy ze wzgledu na dużą liczebność zbioru. Jednak, jeżeli stwierdzimy, że mamy za mało danych uczących to wykorzystamy te rekordy wprowadzając atrybuty wskaźnikowe.

Dodatkowo widzimy, że dotychczas przyznawanymi zniżkami były wartości: {0, 5, 10, 15, 20}


#### deliveris.jsonl - nieużywamy

### Podsumowanie dostępnych danych
Po "konsultacji z klientem" otrzymaliśmy dodatkowe informacje o ocenach poszczególnych produktów oraz dowiedzieliśmy się, że metadane klientów nie są dostępne.
Dane, które otrzymaliśmy nie są idealnie, ale są zdecydowanie wystarczające, żeby spełnić biznesowe kryterium sukcesu. 



## Dane wejściowe/wyjściowe
Ze względu na brak większej liczby metadanych użytkowników, zmuszeni jesteśmy przewidywać sukces tylko na podstawie 'podobieństwa' innych sesji oraz dotychczasowej historii danego użytkownika.

Danymi wejściowymi będzie wiersz opisujący sesję. Będzie on posiadał następujące atrybuty:
- `długość sesji`
- `średnia kwot przeglądanych produktów`
- `mediana kwot przeglądanych produktów`
- `kategorie przeglądanych produktów`
- `miasto użytkownika`
- `płeć użytkownika`
- `zaoferowaną zniżkę`
- `liczbę historycznych sesji danego użytkownika`
- `liczbę historycznych sesji danego użytkownika zakończonych sukcesem`
- `średnia ocena przeglądanych produktów`
- `minimalna --||--`
- `maksymalna --||--`


Wyjściem modelu będzie odpowiedź binarna określająca czy dana sesja zakończy się sukcesem.

Funkcją celu będzie zaproponowana na wykładzie entropia krzyżowa


----

Wnioski wynikające z analizy danych:
* Skuteczność zniżek jest bardzo podobna dla każdej z jej wartości. Jednak zdecydowana większość sesji to sesje bez przyznanej zniżki, przez co szanse na dokonanie zakupu po jej przyznaniu były zdecydowanie niższe. Gdy będziemy przyznawać je częściej bardzo mozliwe, że uda się zwiększyć obrót sklepu. 
* Zdecydowana większość sesji zakończonych zakupem dotyczy klientów, którzy nigdy wcześniej nie kupowali nic w naszym sklepie. Oznacza to, że nie musimy koniecznie koncentrować się na stałych klientach. Warto zachęcać nowych klientów do skorzystania z oferty sklepu.
* Ponad połowę zakupów w naszym sklepie dokonują mężczyźni. To ta grupa zawiera więcej potencjalnych klientów naszego sklepu.
* 5 kategorii produktów najczęścięj odwiedzanych podczas sesji zakończonych sukcesem to: 
    * Gry i konsole
    * Sprzęt RTV             
    * Komputery               
    * Video                   
    * Telefony i akcesoria    
* 5 kategorii produktów najczęścięj odwiedzanych przez kobiety podczas sesji zakończonych sukcesem to: 
    * SprzętRTV                
    * Gry i konsole             
    * Komputery                  
    * Video                      
    * Telewizory i akcesoria  
* 5 kategorii produktów najczęścięj odwiedzanych przez mężczyzn podczas sesji zakończonych sukcesem to: 
    * Gry i konsole         
    * Komputery               
    * Sprzęt RTV             
    * Video                  
    * Telefony i akcesoria 
* Na diagramie korelacji niestety nie można zauważyć żadnych silnych relacji pomiędzy określeniem sesji jako zakończoną sukcesem a innymi pojedynczymi atrybutami. Prawdopodobnie wyjście jest zależne od wielu zależnych od siebie czynników. Możemy zauważyć jedynie zależność pomiędzy cenami i ocenami kleintów w zależności od kategorii produktów, lecz nie jest to użyteczna z perspektywy naszego zadania informacja.